In [41]:

from GUIPlayer import show_board,choose_color,show_id_manager_interface,ask_to_play_another,setup_realtime_board_view
import matplotlib.pyplot as plt
from comments2 import explain_move
# from code_Arm import move_piece,cleanup,get_pict

import torch
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import pickle
import torch
from torchvision import transforms
from PIL import Image
import chess
import chess.engine
import chess.pgn
import os
import numpy as np
import datetime
import chess
from tensorflow.keras.models import load_model
import tkinter as tk
from tkinter import messagebox
import os

In [42]:
engine_path=r"stockfish\stockfish-windows-x86-64-avx2.exe"
engine = chess.engine.SimpleEngine.popen_uci(engine_path)

def evaluate_move( board, move_uci, time=1.0):

    move = chess.Move.from_uci(move_uci)

    if move in board.legal_moves:

        result = engine.analyse(board, chess.engine.Limit(time=time), moves=[move])
        evaluation_score = result["score"].relative.score  # Get evaluation score
        return evaluation_score
    else:
        return None



In [43]:



def get_best_move(board, time_limit: float = 2.0) -> str:

    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        limit = chess.engine.Limit(time=time_limit)
        
        result = engine.play(board, limit)
        best_move = result.move
    
    return best_move 

In [44]:
def load_model_and_tokenizer(id):
    model=None
    tokenizer=None
    if os.path.exists(f"models based on previous games\\players_models\\playerLSTMs{id}.h5"):
        model = load_model(f"models based on previous games\\players_models\\playerLSTMs{id}.h5")


        # Load the tokenizer
        with open(f"models based on previous games\\players_models\\playertokenizer{id}.pkl", "rb") as file:
            tokenizer = pickle.load(file)

    return model,tokenizer 

In [45]:
def lstm_predict(model, tokenizer, sequence):

    sequence_tokenized = tokenizer.texts_to_sequences([sequence])  
    sequence_padded = np.array(sequence_tokenized).reshape(1, -1)  
    
    predictions = model.predict(sequence_padded, verbose=0)  
    predicted_index = np.argmax(predictions)  
    return predicted_index, predictions

In [46]:
def compute_rl_feedback(predicted_move, actual_move,board,c):
    f={0:0.8,1:0.3,2:0.5}
    w1=f[c] if c !=-1 else 0.5
    w2=1-w1 if c !=-1 else 0.5,
    E1=evaluate_move(move_uci=predicted_move,board=board)
    E2=evaluate_move(move_uci=actual_move,board=board)

    S=1 if actual_move==predicted_move else 0

    R=w1*(E2-E1)+w2*S

    improvementFB="Your learning journey is progressing. Even though you're exploring new strategies, keep focusing on the lessons from past games."
    mistakeFB="We can adjust the lessons to match your learning style better. Stay focused and take your time."

    comment= improvementFB if E2>E1 else mistakeFB

    return R,comment

In [47]:
model,tokenizer=load_model_and_tokenizer(1)

In [52]:
print("Tokenizer Vocabulary:", tokenizer.word_index)

Tokenizer Vocabulary: {'c7c5': 1, 'g8f6': 2, 'g1f3': 3, 'd2d4': 4, 'g2g3': 5, 'b8d7': 6, 'd8c7': 7, 'h7h6': 8, 'e1g1': 9, 'a4a5': 10, 'e2e4': 11, 'f2f3': 12, 'h2h3': 13, 'd7d6': 14, 'b1c3': 15, 'g7g6': 16, 'f8g7': 17, 'f1g2': 18, 'd4d5': 19, 'c2c3': 20, 'e7e6': 21, 'd1c2': 22, 'a2a3': 23, 'c2d1': 24, 'e8g8': 25, 'f1h3': 26, 'a8c8': 27, 'f8d8': 28, 'e4e3': 29, 'd2e4': 30, 'b2b4': 31, 'g1g2': 32, 'g2h2': 33, 'b8c6': 34, 'd6d5': 35, 'c1e3': 36, 'f6e4': 37, 'e4c5': 38, 'e3f4': 39, 'f4g5': 40, 'e5e4': 41, 'd7e5': 42, 'g5e3': 43, 'd5d4': 44, 'd1a1': 45, 'b2a1': 46, 'g2f1': 47, 'h6h5': 48, 'd5d6': 49, 'g4g5': 50, 'g2g4': 51, 'h3h2': 52, 'f3f4': 53, 'a2a4': 54, 'e8d7': 55, 'b7b6': 56, 'c8b7': 57, 'b1d2': 58, 'd7d5': 59, 'a3a4': 60, 'f8d6': 61, 'h3g2': 62, 'c5d4': 63, 'f3d4': 64, 'e6e5': 65, 'd4f5': 66, 'd7f8': 67, 'b6a5': 68, 'd5e4': 69, 'd1a4': 70, 'f5e3': 71, 'b7g2': 72, 'f1d1': 73, 'g2a8': 74, 'f8e6': 75, 'a8e4': 76, 'e3g4': 77, 'f6g4': 78, 'c1h6': 79, 'd1d4': 80, 'e4c6': 81, 'a5b4': 82, 'f

In [48]:
board=chess.Board()

In [49]:
predicted_move, prediction = lstm_predict(model,tokenizer,["e2e4"])
print(predicted_move)
print(prediction)

31
[[0.00391036 0.00453988 0.00378553 0.00377563 0.00394035 0.00466974
  0.00394195 0.00369402 0.00430756 0.00317089 0.0043412  0.00493578
  0.00501417 0.00490086 0.00431921 0.00382716 0.00391755 0.00380283
  0.00478781 0.00455622 0.00394878 0.00365831 0.00382936 0.00379845
  0.00383062 0.00396439 0.00448065 0.00399897 0.00335206 0.00504139
  0.00523229 0.00687176 0.00450679 0.00459569 0.00412447 0.00412786
  0.00371296 0.00372748 0.0043292  0.00372843 0.00389703 0.00400743
  0.00331184 0.00371457 0.00415046 0.00458622 0.0043682  0.00513959
  0.00461151 0.00418247 0.00506029 0.00430241 0.00472402 0.00549308
  0.00479016 0.00433997 0.00394023 0.00382666 0.0038856  0.00387571
  0.00360952 0.0037864  0.00401088 0.00391422 0.00398939 0.0040274
  0.00489699 0.00441072 0.00424656 0.00467513 0.0049051  0.00527624
  0.00532183 0.00488162 0.00509034 0.00507232 0.00547916 0.00559135
  0.0062736  0.00578403 0.00527391 0.00457128 0.00548499 0.00566567
  0.0058262  0.00560582 0.00566586 0.00467398 

In [50]:
reward,comment = compute_rl_feedback(predicted_move, "e2e4",board,0) #dealing with it later

TypeError: object of type 'numpy.int64' has no len()